In [ ]:
import os
import keras
import tensorflow as tf
from keras.models import load_model
from keras.layers import Dense, Dropout, Conv1D, Input,MaxPooling1D,Flatten,LeakyReLU,AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam
from group_norm import GroupNormalization
import random
import pandas as pd 
import numpy as np
from Bio import SeqIO
from keras import regularizers
from keras.metrics import binary_accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,recall_score,matthews_corrcoef,roc_curve,roc_auc_score,auc,precision_recall_curve
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau,LearningRateScheduler
import os, sys, copy, getopt, re, argparse
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
from keras import losses
import pickle

from scipy import interp
from sklearn.utils.class_weight import compute_class_weight

from keras.layers import *
from keras import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2 
import keras
from keras.initializers import RandomUniform
import keras.backend as K
from random import shuffle
import itertools 

class CustomModelCheckpoint(keras.callbacks.Callback):
    def __init__(self, model, path):
        self.path = path
        self.model_for_saving = model
    def on_epoch_end(self, epoch, logs=None):
        loss = logs['val_loss']
        print("\nSaving model to : {}".format(self.path.format(epoch=epoch, val_loss=loss)))
        self.model_for_saving.save_weights(self.path.format(epoch=epoch, val_loss=loss), overwrite=True)

def step_decay(epoch):
    lrate=[0.001]*30+[0.0001]*170
    print (lrate[epoch])
    return lrate[epoch]

In [ ]:
def analyze(temp, OutputDir):

    trainning_result, validation_result, testing_result = temp;

    file = open(OutputDir + '/performance.txt', 'w')

    index = 0
    for x in [trainning_result, validation_result, testing_result]:


        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'

        index += 1;

        file.write(title +  'results\n')


        for j in ['sn', 'sp', 'acc', 'MCC','AUC', 'precision', 'F1', 'lossValue']: #,'pre_recall_curve'

            total = []

            for val in x:
                total.append(val[j])
            file.write(j + ' : mean : ' + str(np.mean(total)) + ' std : ' + str(np.std(total))  + '\n')

        file.write('\n\n______________________________\n')
    file.close();

    index = 0

    for x in [trainning_result, validation_result, testing_result]:

        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        
        i = 0

        for val in x:
            tpr = val['tpr']
            fpr = val['fpr']
            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC fold %d (AUC = %0.2f)' % (i+1, roc_auc))

            i += 1

        print;

        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Random', alpha=.8)

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic curve')
        plt.legend(loc="lower right")

        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'

        plt.savefig( OutputDir + '/' + title +'ROC.png')
        plt.close('all');
        
       #************************** Precision Recall Curve*********************************
        i = 0
        prs = []
        pre_aucs = []
        mean_recal= np.linspace(0, 1, 100)
        for val in x:
            pre = val['prec']
            rec = val['reca']
            prs.append(interp(mean_recal, rec, pre))
            prs[-1][0] = 0.0
            p_r_auc = auc(rec, pre)
            pre_aucs.append(p_r_auc)
            plt.plot(rec, pre, lw=1, alpha=0.3,label='PRC fold %d (AUC = %0.2f)' % (i+1, p_r_auc))

            i += 1


        #plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Random', alpha=.8)

        mean_pre = np.mean(prs, axis=0)
        #mean_pre[-1] = 1.0
        mean_auc = auc(mean_recal, mean_pre)
        std_auc = np.std(pre_aucs)
        plt.plot(mean_recal, mean_pre, color='b',
                 label=r'Mean PRC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_pre = np.std(prs, axis=0)
        pre_upper = np.minimum(mean_pre + std_pre, 1)
        pre_lower = np.maximum(mean_pre - std_pre, 0)
        plt.fill_between(mean_recal, pre_lower, pre_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision Recall curve')
        plt.legend(loc="lower right")

        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'
        if index == 3:
            title = 'ind_testing_'

        plt.savefig( OutputDir + '/' + title +'Pre_R_C.png')
        plt.close('all');


        index += 1;

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    
    return out

def calculate(sequence):

    X = []
    dictNum = {'A' : 0, 'T' : 0, 'C' : 0, 'G' : 0};

    for i in range(len(sequence)):

        if sequence[i] in dictNum.keys():
            dictNum[sequence[i]] += 1;
            X.append(dictNum[sequence[i]] / float(i + 1));

    return np.array(X)

#Dinucleotide encoding
# k-mer function
def seq2k_mer(seqs):
    k=2
    sq2=[]
    for seq in seqs:
        sq1=[]
        for x in range(len(seq)-1):
            sq1+=[seq[x:x+k]]
        sq2+=[sq1]
    return sq2
def get_onehot(seq): 
    bases = ['AA', 'AC', 'AG', 'AT', 'AN', 'CA', 'CC', 'CG', 'CT', 'CN', 'GA', 'GC', 'GG', 'GT', 
             'GN', 'TA', 'TC', 'TG', 'TT', 'TN', 'NA','NC', 'NG', 'NT', 'NN']
    X = np.zeros((len(seq)-1, len(bases)))
    #for i, char in enumerate(seq):
    #    X[i, bases.index(char)] = 1
    for i in range(40):
        X[i, bases.index(seq[i:i+2])]=1
    return X

def dataProcessing(seq,key):
    print(len(seq), len(seq[1]))
    bases = ['AA', 'AC', 'AG', 'AT', 'AN', 'CA', 'CC', 'CG', 'CT', 'CN', 'GA', 'GC', 'GG', 'GT', 
             'GN', 'TA', 'TC', 'TG', 'TT', 'TN', 'NA','NC', 'NG', 'NT', 'NN']
    X = np.zeros((len(seq),len(seq[0]), len(bases)))
    for l,s in enumerate(seq):
        for i, char in enumerate(s):
            if char in bases:
                X[l,i, bases.index(char)] = 1
                
    if key == 1:
        lbs = list(np.ones(len(X)))#
    if key == 2:
        lbs=list(np.zeros(len(X)))
    y = np.array(lbs, dtype = np.int32);
    return X, y
    ''''bases = ['A', 'C', 'G', 'T']
    X = np.zeros((len(seq),len(seq[0]), len(bases)))
    for l,s in enumerate(seq):
        for i, char in enumerate(s):
            if char in bases:
                X[l,i, bases.index(char)] = 1
    chem_bases = {'A':[1,1,1], 'C':[0,1,0], 'G':[1,0,0,], 'T':[0,0,1]}
    Z = np.zeros((len(seq),len(seq[0]), 3))
    for l,s in enumerate(seq):
        for i, char in enumerate(s):
            if char in chem_bases:
                Z[l][i]=(chem_bases[char])
                
    all_features=np.concatenate([X,Z],axis=2)
    if key == 1:
        lbs = list(np.ones(len(X)))#
    if key == 2:
        lbs=list(np.zeros(len(X)))
    y = np.array(lbs, dtype = np.int32)
 
    return X, y #(n, 34, 4), (n,)'''''

def prepareData(path):
    all_seq = []
    for seq_record in SeqIO.parse(path, "fasta"):
        all_seq.append(str(seq_record.seq))
    pos_seq = []
    neg_seq = []
    for i in range(len(all_seq)):
        if(i < (len(all_seq)/2)):
            pos_seq.append(all_seq[i])
        else:
            neg_seq.append(all_seq[i])
    a=1
    b=2
    
    Positive_X, Positive_y = dataProcessing(seq2k_mer(pos_seq),a)
    Negitive_X, Negitive_y = dataProcessing(seq2k_mer(neg_seq),b)

    return Positive_X, Positive_y, Negitive_X, Negitive_y

def shuffleData(X, y):
    index = [i for i in range(len(X))]
    random.shuffle(index)
    X = X[index]
    y = y[index]
    return X, y;

def getMode():

    input_shape = (40,25)

    inputs = Input(shape = input_shape)

    convLayer = Conv1D(filters = 32, kernel_size = 5,activation = 'elu',input_shape = input_shape, kernel_regularizer = regularizers.l2(1e-4), bias_regularizer = regularizers.l2(1e-4))(inputs)
    normalizationLayer = GroupNormalization(groups = 4,axis=-1)(convLayer)
    #normalizationLayer = BatchNormalization()(convLayer);
    poolingLayer = MaxPooling1D(pool_size = 4, strides=2)(normalizationLayer)
    dropoutLayer0 = Dropout(0.25)(poolingLayer)
    
    convLayer2 = Conv1D(filters = 32, kernel_size = 5,activation = 'elu',kernel_regularizer = regularizers.l2(1e-4), bias_regularizer = regularizers.l2(1e-4))(dropoutLayer0)
    normalizationLayer2 = GroupNormalization(groups = 4,axis=-1)(convLayer2)
    #normalizationLayer = BatchNormalization()(convLayer);
    poolingLayer2 = MaxPooling1D(pool_size = 4, strides=2)(normalizationLayer2)
    dropoutLayer = Dropout(0.25)(poolingLayer2)
    
    flattenLayer = Flatten()(dropoutLayer)

    denseLayer = Dense(32, activation = 'elu',kernel_regularizer = regularizers.l2(1e-4),bias_regularizer = regularizers.l2(1e-4))(flattenLayer)
    outLayer = Dense(1, activation='sigmoid')(denseLayer)

    model = Model(inputs = inputs, outputs = outLayer)
    model.compile(loss='binary_crossentropy', optimizer= SGD(momentum = 0.95, lr = 0.005),metrics=[binary_accuracy])

    print(model.summary())

    return model


def calculateScore(X, y, model):
    
    score = model.evaluate(X,y)
    pred_y = model.predict(X)
    accuracy = score[1]
    

    tempLabel = np.zeros(shape = y.shape, dtype=np.int32)

    for i in range(len(y)):
        if pred_y[i] < 0.5:
            tempLabel[i] = 0;
        else:
            tempLabel[i] = 1;
    confusion = confusion_matrix(y, tempLabel)
    TN, FP, FN, TP = confusion.ravel()

    sensitivity = recall_score(y, tempLabel)
    specificity = TN / float(TN+FP)
    MCC = matthews_corrcoef(y, tempLabel)

    F1Score = (2 * TP) / float(2 * TP + FP + FN)
    precision = TP / float(TP + FP)
    recall = TP/float (TP+FN)
    pred_y = pred_y.reshape((-1, ))

    ROCArea = roc_auc_score(y, pred_y)
    fpr, tpr, thresholds = roc_curve(y, pred_y)
    lossValue = None;
    
    pre, rec, threshlds = precision_recall_curve(y, pred_y)
    pre = np.fliplr([pre])[0]  #so the array is increasing (you won't get negative AUC)
    rec = np.fliplr([rec])[0]  
    AUC_prec_rec = np.trapz(rec,pre)
    AUC_prec_rec = abs(AUC_prec_rec)
    
    print(y.shape)
    print(pred_y.shape)

    y_true = tf.convert_to_tensor(y, np.float32)
    y_pred = tf.convert_to_tensor(pred_y, np.float32)

    with tf.Session():
        lossValue = losses.binary_crossentropy(y_true, y_pred).eval()  #,'pre_recall_curve':AUC_prec_rec
    plt.show()
    return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea,'precision' : precision, 'F1' : F1Score, 'fpr' : fpr, 'tpr' : tpr, 'thresholds' : thresholds, 'lossValue' : lossValue,'pre_recall_curve':AUC_prec_rec,'prec':pre,'reca':rec}
def test_data_prepro(ind_test):
    Positive_X, Positive_y, Negitive_X, Negitive_y = prepareData(ind_test)
    test_X = np.concatenate((Positive_X,Negitive_X))
    test_y = np.concatenate((Positive_y,Negitive_y))
    return test_X, test_y
def funciton(All_data, OutputDir, folds):

    Positive_X, Positive_y, Negitive_X, Negitive_y = prepareData(All_data)
    
    random.shuffle(Positive_X)
    random.shuffle(Negitive_X)

    Positive_X_Slices = chunkIt(Positive_X, folds)
    Positive_y_Slices = chunkIt(Positive_y, folds)

    Negative_X_Slices = chunkIt(Negitive_X, folds)
    Negative_y_Slices = chunkIt(Negitive_y, folds)

    trainning_result = []
    validation_result = []
    testing_result = []
    #ind_testing_result = []
    
    for test_index in range(folds):

        test_X = np.concatenate((Positive_X_Slices[test_index],Negative_X_Slices[test_index]))
        test_y = np.concatenate((Positive_y_Slices[test_index],Negative_y_Slices[test_index]))
        
        validation_index = (test_index+1) % folds;

        valid_X = np.concatenate((Positive_X_Slices[validation_index],Negative_X_Slices[validation_index]))
        valid_y = np.concatenate((Positive_y_Slices[validation_index],Negative_y_Slices[validation_index]))

        start = 0;

        for val in range(0, folds):
            if val != test_index and val != validation_index:
                start = val;
                break;

        train_X = np.concatenate((Positive_X_Slices[start],Negative_X_Slices[start]))
        train_y = np.concatenate((Positive_y_Slices[start],Negative_y_Slices[start]))

        for i in range(0, folds):
            if i != test_index and i != validation_index and i != start:
                tempX = np.concatenate((Positive_X_Slices[i],Negative_X_Slices[i]))
                tempy = np.concatenate((Positive_y_Slices[i],Negative_y_Slices[i]))

                
                train_X = np.concatenate((train_X, tempX))
                train_y = np.concatenate((train_y, tempy))

    
        test_X, test_y = shuffleData(test_X,test_y);
        valid_X,valid_y = shuffleData(valid_X,valid_y)
        train_X,train_y = shuffleData(train_X,train_y);
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'x_test',test_X)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'y_test',test_y)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'valid_X',valid_X)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'valid_y',valid_y)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'x_train',train_X)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'y_train',train_y)
        
        #ind_test = 'H_200.txt'
        #X_test,y_test=test_data_prepro(ind_test)
        #ind_test_X, ind_test_y = shuffleData(X_test,y_test)
        #np.save('/home/waleed/pseu_RNA/outputs/chunk_folds/'+str(test_index)+'_'+'ind_test_y',ind_test_X)
        #np.save('/home/waleed/pseu_RNA/outputs/chunk_folds/'+str(test_index)+'_'+'ind_test_y',ind_test_y)
        ###test####
        
        model = getMode()
        
        result_folder = OutputDir
        if not os.path.exists(result_folder):
            os.makedirs(result_folder)
        model_results_folder=result_folder
        
        #best_weights = model_results_folder + 'best_weights.h5'
        
        early_stopping = EarlyStopping(monitor='val_loss', patience= 10, restore_best_weights=True)
        model_check = ModelCheckpoint(filepath = OutputDir + "/model" + str(test_index+1) +".h5",mode='min', monitor = 'val_loss', save_best_only=True)#, save_weights_only=True
        reduct_L_rate = ReduceLROnPlateau(monitor='val_loss',factor=0.01, patience=10)
        lrate = LearningRateScheduler(step_decay)
        cbacks = [model_check,reduct_L_rate,early_stopping]
        
        history = model.fit(train_X, train_y, batch_size = 16, epochs =200,validation_data = (valid_X, valid_y),callbacks = cbacks);
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        
        model=load_model('C:/Users/NSCL/N4-methylctosine//outputs//model'+str(test_index+1)+'.h5')
        
        trainning_result.append(calculateScore(train_X, train_y, model))
        validation_result.append(calculateScore(valid_X, valid_y, model))
        testing_result.append(calculateScore(test_X, test_y, model))
        #ind_testing_result.append(calculateScore(ind_test_X, ind_test_y, model));

    temp_dict = (trainning_result, validation_result, testing_result)
    analyze(temp_dict, OutputDir)

In [ ]:
All_data = 'D.melanogaster.fasta.txt' 
OutputDir = 'C:/Users/NSCL/N4-methylctosine//outputs/'
funciton(All_data, OutputDir, 10)

In [ ]:
from keras.models import load_model
from keras.layers import Dense, Dropout, Conv1D, Input,MaxPooling1D,Flatten,LeakyReLU,AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam
from group_norm import GroupNormalization
import random
import pandas as pd 
import numpy as np
from Bio import SeqIO
from keras import regularizers
from keras.metrics import binary_accuracy
from sklearn.metrics import confusion_matrix,recall_score,matthews_corrcoef,roc_curve,roc_auc_score,auc,precision_recall_curve
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau,LearningRateScheduler
import os, sys, copy, getopt, re, argparse
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
from keras import losses
import pickle

from scipy import interp
from sklearn.utils.class_weight import compute_class_weight

from keras.layers import *
from keras import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2 
import keras
from keras.initializers import RandomUniform
import keras.backend as K
from random import shuffle
import itertools 

class CustomModelCheckpoint(keras.callbacks.Callback):
    def __init__(self, model, path):
        self.path = path
        self.model_for_saving = model
    def on_epoch_end(self, epoch, logs=None):
        loss = logs['val_loss']
        print("\nSaving model to : {}".format(self.path.format(epoch=epoch, val_loss=loss)))
        self.model_for_saving.save_weights(self.path.format(epoch=epoch, val_loss=loss), overwrite=True)

def step_decay(epoch):
    lrate=[0.001]*30+[0.0001]*170
    print (lrate[epoch])
    return lrate[epoch]


In [ ]:
def analyze(temp, OutputDir):

    trainning_result, validation_result, testing_result = temp;

    file = open(OutputDir + '/performance.txt', 'w')

    index = 0
    for x in [trainning_result, validation_result, testing_result]:


        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'

        index += 1;

        file.write(title +  'results\n')


        for j in ['sn', 'sp', 'acc', 'MCC','AUC', 'precision', 'F1', 'lossValue']: #,'pre_recall_curve'

            total = []

            for val in x:
                total.append(val[j])
            file.write(j + ' : mean : ' + str(np.mean(total)) + ' std : ' + str(np.std(total))  + '\n')

        file.write('\n\n______________________________\n')
    file.close();

    index = 0

    for x in [trainning_result, validation_result, testing_result]:

        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        
        i = 0

        for val in x:
            tpr = val['tpr']
            fpr = val['fpr']
            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC fold %d (AUC = %0.2f)' % (i+1, roc_auc))

            i += 1

        print;

        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Random', alpha=.8)

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic curve')
        plt.legend(loc="lower right")

        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'

        plt.savefig( OutputDir + '/' + title +'ROC.png')
        plt.close('all');
        
       #************************** Precision Recall Curve*********************************
        i = 0
        prs = []
        pre_aucs = []
        mean_recal= np.linspace(0, 1, 100)
        for val in x:
            pre = val['prec']
            rec = val['reca']
            prs.append(interp(mean_recal, rec, pre))
            prs[-1][0] = 0.0
            p_r_auc = auc(rec, pre)
            pre_aucs.append(p_r_auc)
            plt.plot(rec, pre, lw=1, alpha=0.3,label='PRC fold %d (AUC = %0.2f)' % (i+1, p_r_auc))

            i += 1


        #plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Random', alpha=.8)

        mean_pre = np.mean(prs, axis=0)
        #mean_pre[-1] = 1.0
        mean_auc = auc(mean_recal, mean_pre)
        std_auc = np.std(pre_aucs)
        plt.plot(mean_recal, mean_pre, color='b',
                 label=r'Mean PRC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_pre = np.std(prs, axis=0)
        pre_upper = np.minimum(mean_pre + std_pre, 1)
        pre_lower = np.maximum(mean_pre - std_pre, 0)
        plt.fill_between(mean_recal, pre_lower, pre_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision Recall curve')
        plt.legend(loc="lower right")

        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
        if index == 2:
            title = 'testing_'
        if index == 3:
            title = 'ind_testing_'

        plt.savefig( OutputDir + '/' + title +'Pre_R_C.png')
        plt.close('all');


        index += 1;

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    
    return out

def calculate(sequence):

    X = []
    dictNum = {'A' : 0, 'T' : 0, 'C' : 0, 'G' : 0};

    for i in range(len(sequence)):

        if sequence[i] in dictNum.keys():
            dictNum[sequence[i]] += 1;
            X.append(dictNum[sequence[i]] / float(i + 1));

    return np.array(X)

#nucleotide chemical properties and frequency encoding
def cal(c,cb,i):
    bases ={'A':[1,1,1], 'C':[0,1,0], 'G':[1,0,0,], 'T':[0,0,1]}
    p=[]
    p=bases[c]
    p.append(np.round(cb/float(i+1),2))
    return(p)
def calculate(s):
    p=f=list()
    cba=cbc=cbt=cbg=0
    for i,c in enumerate(s):
        if c=='A':
            cba+=1
            p=cal(c,cba,i)
        elif c=='T':
            cbt+=1
            p=cal(c,cbt,i)
        elif c=='C':
            cbc+=1
            p=cal(c,cbc,i)
        elif c=='G':
            cbg+=1
            p=cal(c,cbg,i)
        else:
            p=[0,0,0,0]
        f.append(p)
    return(f)
# k-mer function
def seq2k_mer(seqs):
    k=2
    sq2=[]
    for seq in seqs:
        sq1=[]
        for x in range(len(seq)-1):
            sq1+=[seq[x:x+k]]
        sq2+=[sq1]
    return sq2

def dataProcessing(seq,key):
    #bases = ['A', 'C', 'G', 'T']
    bases = ['AA', 'AT' ,'AC', 'AG', 'TA' ,'TT', 'TC' ,'TG', 'CA', 'CT', 'CC', 'CG', 'GA', 'GT' ,'GC', 'GG']
    X = np.zeros((len(seq),len(seq[0]), len(bases)))
    for l,s in enumerate(seq):
        for i, char in enumerate(s):
            if char in bases:
                X[l,i, bases.index(char)] = 1
    
    if key == 1:
        lbs = list(np.ones(len(X)))#
    if key == 2:
        lbs=list(np.zeros(len(X)))
    y = np.array(lbs, dtype = np.int32)
 
    return X, y #(n, 34, 4), (n,)

def prepareData(path):
    all_seq = []
    for seq_record in SeqIO.parse(path, "fasta"):
        all_seq.append(str(seq_record.seq))
    pos_seq = []
    neg_seq = []
    for i in range(len(all_seq)):
        if(i < (len(all_seq)/2)):
            pos_seq.append(all_seq[i])
        else:
            neg_seq.append(all_seq[i])
    a=1
    b=2
    
    Positive_X, Positive_y = dataProcessing(seq2k_mer(pos_seq),a);
    Negitive_X, Negitive_y = dataProcessing(seq2k_mer(neg_seq),b);

    return Positive_X, Positive_y, Negitive_X, Negitive_y

def shuffleData(X, y):
    index = [i for i in range(len(X))]
    random.shuffle(index)
    X = X[index]
    y = y[index]
    return X, y;

def getMode():

    input_shape = (40,16)

    inputs = Input(shape = input_shape)

    convLayer = Conv1D(filters = 8, kernel_size = 3,padding='same',kernel_regularizer = regularizers.l2(1e-3),bias_regularizer = regularizers.l2(1e-4),activation = 'relu',input_shape = input_shape)(inputs);
    #normalizationLayer = GroupNormalization(groups = 4,axis=-1)(convLayer)
    normalizationLayer=BatchNormalization()(convLayer)
    #poolingLayer = AveragePooling1D(pool_size = 2,padding='same')(normalizationLayer)
    convLayer2 = Conv1D(filters = 8, kernel_size = 3,padding='same',kernel_regularizer = regularizers.l2(1e-3),bias_regularizer = regularizers.l2(1e-4),activation = 'relu',input_shape = input_shape)(normalizationLayer)

    flattenLayer = Flatten()(convLayer2)
    denseLayer = Dense(5, activation = 'relu',kernel_regularizer = regularizers.l2(1e-3),bias_regularizer = regularizers.l2(1e-4))(flattenLayer)
    dropoutLayer = Dropout(0.3)(denseLayer)
    outLayer = Dense(1, activation='sigmoid')(dropoutLayer)
    model = Model(inputs = inputs, outputs = outLayer)
    model.compile(loss='binary_crossentropy', optimizer= Adam(lr=0.001), metrics=['accuracy']); #SGD(lr = 0.005, momentum=0.95)

    print(model.summary())

    return model


def calculateScore(X, y, model):
    
    score = model.evaluate(X,y)
    pred_y = model.predict(X)
    print(score)

    accuracy = score[1];

    tempLabel = np.zeros(shape = y.shape, dtype=np.int32)

    for i in range(len(y)):
        if pred_y[i] < 0.5:
            tempLabel[i] = 0;
        else:
            tempLabel[i] = 1;
    confusion = confusion_matrix(y, tempLabel)
    TN, FP, FN, TP = confusion.ravel()

    sensitivity = recall_score(y, tempLabel)
    specificity = TN / float(TN+FP)
    MCC = matthews_corrcoef(y, tempLabel)

    F1Score = (2 * TP) / float(2 * TP + FP + FN)
    precision = TP / float(TP + FP)
    recall = TP/float (TP+FN)
    pred_y = pred_y.reshape((-1, ))

    ROCArea = roc_auc_score(y, pred_y)
    fpr, tpr, thresholds = roc_curve(y, pred_y)
    lossValue = None;
    
    pre, rec, threshlds = precision_recall_curve(y, pred_y)
    pre = np.fliplr([pre])[0]  #so the array is increasing (you won't get negative AUC)
    rec = np.fliplr([rec])[0]  
    AUC_prec_rec = np.trapz(rec,pre)
    AUC_prec_rec = abs(AUC_prec_rec)
    
    print(y.shape)
    print(pred_y.shape)

    y_true = tf.convert_to_tensor(y, np.float32)
    y_pred = tf.convert_to_tensor(pred_y, np.float32)

    with tf.Session():
        lossValue = losses.binary_crossentropy(y_true, y_pred).eval()  #,'pre_recall_curve':AUC_prec_rec
    plt.show()
    return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea,'precision' : precision, 'F1' : F1Score, 'fpr' : fpr, 'tpr' : tpr, 'thresholds' : thresholds, 'lossValue' : lossValue,'pre_recall_curve':AUC_prec_rec,'prec':pre,'reca':rec}
def test_data_prepro(ind_test):
    Positive_X, Positive_y, Negitive_X, Negitive_y = prepareData(ind_test)
    test_X = np.concatenate((Positive_X,Negitive_X))
    test_y = np.concatenate((Positive_y,Negitive_y))
    return test_X, test_y
def funciton(All_data, OutputDir, folds):

    Positive_X, Positive_y, Negitive_X, Negitive_y = prepareData(All_data)
    
    random.shuffle(Positive_X);
    random.shuffle(Negitive_X);

    Positive_X_Slices = chunkIt(Positive_X, folds);
    Positive_y_Slices = chunkIt(Positive_y, folds);

    Negative_X_Slices = chunkIt(Negitive_X, folds);
    Negative_y_Slices = chunkIt(Negitive_y, folds);

    trainning_result = []
    validation_result = []
    testing_result = []
    #ind_testing_result = []
    
    for test_index in range(folds):

        test_X = np.concatenate((Positive_X_Slices[test_index],Negative_X_Slices[test_index]))
        test_y = np.concatenate((Positive_y_Slices[test_index],Negative_y_Slices[test_index]))
        
        validation_index = (test_index+1) % folds;

        valid_X = np.concatenate((Positive_X_Slices[validation_index],Negative_X_Slices[validation_index]))
        valid_y = np.concatenate((Positive_y_Slices[validation_index],Negative_y_Slices[validation_index]))

        start = 0;

        for val in range(0, folds):
            if val != test_index and val != validation_index:
                start = val;
                break;

        train_X = np.concatenate((Positive_X_Slices[start],Negative_X_Slices[start]))
        train_y = np.concatenate((Positive_y_Slices[start],Negative_y_Slices[start]))

        for i in range(0, folds):
            if i != test_index and i != validation_index and i != start:
                tempX = np.concatenate((Positive_X_Slices[i],Negative_X_Slices[i]))
                tempy = np.concatenate((Positive_y_Slices[i],Negative_y_Slices[i]))

                
                train_X = np.concatenate((train_X, tempX))
                train_y = np.concatenate((train_y, tempy))

    
        test_X, test_y = shuffleData(test_X,test_y);
        valid_X,valid_y = shuffleData(valid_X,valid_y)
        train_X,train_y = shuffleData(train_X,train_y);
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'x_test',test_X)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'y_test',test_y)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'valid_X',valid_X)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'valid_y',valid_y)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'x_train',train_X)
        np.save('C:/Users/NSCL/N4-methylctosine//outputs/chunk_folds/'+str(test_index)+'_'+'y_train',train_y)
        
        #ind_test = 'H_200.txt'
        #X_test,y_test=test_data_prepro(ind_test)
        #ind_test_X, ind_test_y = shuffleData(X_test,y_test)
        #np.save('/home/waleed/pseu_RNA/outputs/chunk_folds/'+str(test_index)+'_'+'ind_test_y',ind_test_X)
        #np.save('/home/waleed/pseu_RNA/outputs/chunk_folds/'+str(test_index)+'_'+'ind_test_y',ind_test_y)
        ###test####
        
        model = getMode()
        
        result_folder = OutputDir
        if not os.path.exists(result_folder):
            os.makedirs(result_folder)
        model_results_folder=result_folder
        
        #best_weights = model_results_folder + 'best_weights.h5'
        
        early_stopping = EarlyStopping(monitor='val_loss', patience= 20, restore_best_weights=True)
        model_check = ModelCheckpoint(filepath = OutputDir + "/model" + str(test_index+1) +".h5",mode='max', monitor = 'val_acc', save_best_only=True)#, save_weights_only=True
        reduct_L_rate = ReduceLROnPlateau(monitor='val_loss',factor=0.01, patience=10)
        lrate = LearningRateScheduler(step_decay)
        cbacks = [model_check,reduct_L_rate,early_stopping]
        
        history = model.fit(train_X, train_y, batch_size = 32, epochs =200,validation_data = (valid_X, valid_y),callbacks = cbacks);
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        
        model=load_model('C:/Users/NSCL/N4-methylctosine//outputs//model'+str(test_index+1)+'.h5')
        
        trainning_result.append(calculateScore(train_X, train_y, model))
        validation_result.append(calculateScore(valid_X, valid_y, model))
        testing_result.append(calculateScore(test_X, test_y, model))
        #ind_testing_result.append(calculateScore(ind_test_X, ind_test_y, model));

    temp_dict = (trainning_result, validation_result, testing_result)
    analyze(temp_dict, OutputDir)

In [ ]:
All_data = 'G.subterraneus.fasta.txt' 
OutputDir = 'C:/Users/NSCL/N4-methylctosine//outputs/'
funciton(All_data, OutputDir, 10)